<a href="https://colab.research.google.com/github/mohzary/python-deep-learning-f19/blob/master/M2ICP5/M2ICP5-1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## In class programming:
1. Save the model and use the saved model to predict on new text data (ex, “A lot of good things are happening. We are respected again throughout the world, and that's a great thing.@realDonaldTrump”).

2. Apply GridSearchCV on the source code provided in the class

In [0]:
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import Dense, Embedding, LSTM, SpatialDropout1D
from matplotlib import pyplot
from sklearn.model_selection import train_test_split
from keras.utils.np_utils import to_categorical
import re
import io
from google.colab import files
from sklearn.preprocessing import LabelEncoder

In [0]:
# To upload data file from local folder to colab
data_file = files.upload()


Saving Sentiment.csv to Sentiment.csv


In [0]:
data = pd.read_csv(io.StringIO(data_file['Sentiment.csv'].decode('utf-8')))


In [0]:
data.head(5)

,id,candidate,candidate_confidence,relevant_yn,relevant_yn_confidence,sentiment,sentiment_confidence,subject_matter,subject_matter_confidence,candidate_gold,name,relevant_yn_gold,retweet_count,sentiment_gold,subject_matter_gold,text,tweet_coord,tweet_created,tweet_id,tweet_location,user_timezone
0,1,No candidate mentioned,1.0,yes,1.0,Neutral,0.6578,None of the above,1.0000,NaN,I_Am_Kenzi,NaN,5,NaN,NaN,RT @NancyLeeGrahn: How did everyone feel about...,NaN,2015-08-07 09:54:46 -0700,629697200650592256,NaN,Quito
1,2,Scott Walker,1.0,yes,1.0,Positive,0.6333,None of the above,1.0000,NaN,PeacefulQuest,NaN,26,NaN,NaN,RT @ScottWalker: Didn't catch the full #GOPdeb...,NaN,2015-08-07 09:54:46 -0700,629697199560069120,NaN,NaN
2,3,No candidate mentioned,1.0,yes,1.0,Neutral,0.6629,None of the above,0.6629,NaN,PussssyCroook,NaN,27,NaN,NaN,RT @TJMShow: No mention of Tamir Rice and the ...,NaN,2015-08-07 09:54:46 -0700,629697199312482304,NaN,NaN
3,4,No candidate mentioned,1.0,yes,1.0,Positive,1.0000,None of the above,0.7039,NaN,MattFromTexas31,NaN,138,NaN,NaN,RT @RobGeorge: That Carly Fiorina is trending ...,NaN,2015-08-07 09:54:45 -0700,629697197118861312,Texas,Central Time (US & Canada)
4,5,Donald Trump,1.0,yes,1.0,Positive,0.7045,None of the above,1.0000,NaN,sharonDay5,NaN,156,NaN,NaN,RT @DanScavino: #GOPDebate w/ @realDonaldTrump...,NaN,2015-08-07 09:54:45 -0700,629697196967903232,NaN,Arizona


In [0]:
# Keeping only the neccessary columns
data = data[['text','sentiment']]

# to return the lowercased string from the given string. It converts all uppercase characters to lowercase. If no uppercase characters exist, it returns the original string.
data['text'] = data['text'].apply(lambda x: x.lower())

data['text'] = data['text'].apply((lambda x: re.sub('[^a-zA-z0-9\s]', '', x)))

for idx, row in data.iterrows():
    row[0] = row[0].replace('rt', ' ')


In [0]:
data.head(5)

,text,sentiment
0,nancyleegrahn how did everyone feel about th...,Neutral
1,scottwalker didnt catch the full gopdebate l...,Positive
2,tjmshow no mention of tamir rice and the gop...,Neutral
3,robgeorge that carly fiorina is trending ho...,Positive
4,danscavino gopdebate w realdonaldtrump deliv...,Positive


In [0]:
max_fatures = 2000
tokenizer = Tokenizer(num_words=max_fatures, split=' ')
tokenizer.fit_on_texts(data['text'].values)
X = tokenizer.texts_to_sequences(data['text'].values)

In [0]:
X = pad_sequences(X)

embed_dim = 128
lstm_out = 196

In [0]:
def createmodel():
    model = Sequential()
    model.add(Embedding(max_fatures, embed_dim,input_length = X.shape[1]))
    model.add(LSTM(lstm_out, dropout=0.2, recurrent_dropout=0.2))
    model.add(Dense(3,activation='softmax'))
    model.compile(loss = 'categorical_crossentropy', optimizer='adam',metrics = ['accuracy'])
    return model
#print(model.summary())

In [0]:
labelencoder = LabelEncoder()
integer_encoded = labelencoder.fit_transform(data['sentiment'])
y = to_categorical(integer_encoded)

In [0]:
X_train, X_test, Y_train, Y_test = train_test_split(X,y, test_size = 0.33, random_state = 42)


In [0]:
from keras.callbacks import TensorBoard
tbCallBack= TensorBoard(log_dir='./GraphICP5-1', histogram_freq=0,write_graph=True, write_images=True) 
batch_size = 32
model = createmodel()
model.fit(X_train, Y_train, validation_data=(X_test, Y_test), epochs = 1, batch_size=batch_size, verbose = 2, callbacks=[tbCallBack])

Train on 9293 samples, validate on 4578 samples
Epoch 1/1
 - 28s - loss: 0.8333 - acc: 0.6423 - val_loss: 0.7586 - val_acc: 0.6645


In [0]:
score,acc = model.evaluate(X_test,Y_test,verbose=2,batch_size=batch_size)
print("Loss Score: ", (score))
print("Accuracy: %.2f%%" % (acc*100))
print(model.metrics_names)

Loss Score:  0.7586365605205259
Accuracy: 66.45%
['loss', 'acc']


In [0]:
# to Save the model 

model.save('smodel.h5')
from keras.models import load_model 
savedmodel = load_model('smodel.h5')



In [230]:
# To use the saved model to predict on new text data:
import numpy as np

myString = ["A lot of good things are happening. We are respected again throughout the world, and that's a great thing.@realDonaldTrump"]


#myString = myString.lower()

#myString = re.sub('[^a-zA-z0-9\s]', '', myString)





max_fatures = 2000
tokenizer = Tokenizer(num_words=max_fatures, split=' ')
tokenizer.fit_on_texts(myString)
myStringX = tokenizer.texts_to_sequences(myString )
myStringX = pad_sequences(myStringX, maxlen=28)

print(myStringX.shape)




(1, 28)


In [233]:

predictionResult = savedmodel.predict_classes(myStringX)
modelClass = ['positive', 'Negative']
print("Predicted Class is: ", (predictionResult))

print(modelClass[predictionResult[0]])

Predicted Class is:  [0]
positive
